## Fix Column Headers

In [1]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Fix Headers") \
    .master("local[2]") \
    .getOrCreate()

spark

In [10]:
%%sh 

more "dataset/students.csv"

::::::::::::::
dataset/students.csv
::::::::::::::
﻿First Name,Last Name,Dept Name,Dept ID,Salary
Amit,Kumar,Physics,PHY,30000
Anwar,Ansari,Chemistry,CHEM,30000
Rita,,Maths,MATH,39000
Deepak,Chaturvedi,Biology,BIO,30000

In [16]:
# Python function to read the column name and fix the space with underscore "_"
from pyspark.sql import DataFrame

def fix_header(df: DataFrame) -> list:
    fixed_col_list: list = []
    for col in df.columns:
        fixed_col_list.append(f"`{str(col).strip()}` as {str(col).strip().replace(' ','_').lower()}")
        
    return fixed_col_list

In [22]:
# Read the CSV file with malformed header
raw_df = spark.read.format("csv").option("header", True).load("dataset/students.csv")
raw_df.printSchema()
raw_df.show()

root
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Dept Name: string (nullable = true)
 |-- Dept ID: string (nullable = true)
 |-- Salary: string (nullable = true)

+----------+----------+---------+-------+------+
|First Name| Last Name|Dept Name|Dept ID|Salary|
+----------+----------+---------+-------+------+
|      Amit|     Kumar|  Physics|    PHY| 30000|
|     Anwar|    Ansari|Chemistry|   CHEM| 30000|
|      Rita|      null|    Maths|   MATH| 39000|
|    Deepak|Chaturvedi|  Biology|    BIO| 30000|
+----------+----------+---------+-------+------+



In [21]:
# Create a new dataframe with fixed column names
fixed_headers = fix_header(df = raw_df)
print(fixed_headers)

# Apply to create the new dataframe
fixed_df = df.selectExpr(fixed_headers)
fixed_df.printSchema()
fixed_df.show()

['`First Name` as first_name', '`Last Name` as last_name', '`Dept Name` as dept_name', '`Dept ID` as dept_id', '`Salary` as salary']
root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- salary: string (nullable = true)

+----------+----------+---------+-------+------+
|first_name| last_name|dept_name|dept_id|salary|
+----------+----------+---------+-------+------+
|      Amit|     Kumar|  Physics|    PHY| 30000|
|     Anwar|    Ansari|Chemistry|   CHEM| 30000|
|      Rita|      null|    Maths|   MATH| 39000|
|    Deepak|Chaturvedi|  Biology|    BIO| 30000|
+----------+----------+---------+-------+------+

